<a href="https://colab.research.google.com/github/keirace/artificial-guacamole/blob/master/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Nov 21 09:34:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 ฝึกงาน
 เกี่ยวกับเรา
 1080p.mov
'12. รูปแบบบทคัดย่อ 63 พิศาล.docx'
'ปกเเลป (1).docx'
 การทดลอง1p1.docx
 การทดลอง1p1.pdf
 การทดลอง1p2.docx
 การทดลอง1p2.pdf
'2020-07-17 05-46-59.mp4'
 23in19020451_20191218090331PM.pdf
 406714_pplant.pdf
 เเลป4P1.pdf
 แลป4P2ซุปเปอร์.docx
 แลป4P2ซุปเปอร์.pdf
 5d9e4795a3414877bd6cf04a5c8cf0ce.pdf
 6010613401cn330.pdf
 6010613401-grid-hw.zip
'การทดลองที่ 6.docx'
'การทดลองที่ 6.pdf'
 รายงานการประชุมครั้งที่6.pdf
'เเลป 7.docx'
 สรุปผลการทดลองเเลป9.docx
 ALU_lab7.circ
 api
 api.rar
 application
 blender
 Blink.ino
'BOP (1).pdf'
 BOP.pdf
 Breakout.rar
 byenior
 Chest-X-ray-Project_final.pptx
 ci
'Classic Resume.docx'
 Classroom
 client.py
 CN203-lec02a-sorting2.pdf
 CN203-lec02b-sorting2.pdf
 CN203-lec03a-AVL-trees.pdf
 CN203-lec03b-234-trees.pdf
 CN203-lec04-hashtables.pdf
 CN203-lec05.2-graphs-BFS.pdf
 CN203-lec05.3-graphs-DFS.pdf
 CN203-lec05.4-graphs-appDFS.pdf
 CN203-lec05.6-graphs-Dijkstra.pdf
 CN203-lec05.7-graphs-BellmanFord.p

**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14449 (delta 14), reused 20 (delta 12), pack-reused 14421
Receiving objects: 100% (14449/14449), 13.18 MiB | 22.49 MiB/s, done.
Resolving deltas: 100% (9820/9820), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [6]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
# !mkdir "/content/drive/My Drive/yolov3"

In [12]:
!echo -e "bad\ngood" > data/obj.names
!echo -e 'classes= 2\ntrain = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-11-21 09:37:10--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  3.85MB/s    in 65s     

2020-11-21 09:38:16 (2.37 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [13]:
# !unzip '/content/gdrive/My Drive/yolov3/world food edit.zip' -d data/obj
!unrar x '/content/train.rar' 'data/obj'


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/train.rar

Creating    data/obj/good                                             OK
Extracting  data/obj/good/20200609_103333.jpg                              3%  OK 
Extracting  data/obj/good/20200609_103333.txt                              3%  OK 
Extracting  data/obj/good/20200609_103425.jpg                              7%  OK 
Extracting  data/obj/good/20200609_103425.txt                              7%  OK 
Extracting  data/obj/good/20200623_153714.jpg                             11%  OK 
Extracting  data/obj/good/20200623_153714.txt                             11%  OK 
Extracting  data/obj/good/black_good_2020-11-18_01-34-53.785166.jpg       12%  OK 
Extracting  data/obj/good/black_good_2020-11-18_01-34-53.785166.txt       12%  OK 
Extracting  data/obj/good/black_good_2020-11-18_01-34-55.109395.jpg       12%

In [14]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"/content/darknet/data/obj/*/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

1 0.479911 0.455357 0.691964 0.348214

['1', '0.479911', '0.455357', '0.691964', '0.348214']
0 /content/darknet/data/obj/good/black_good_2020-11-18_01-35-37.707507.txt
0 0.479911 0.455357 0.691964 0.348214
1 0.506696 0.361607 0.852679 0.428571

['1', '0.506696', '0.361607', '0.852679', '0.428571']
1 /content/darknet/data/obj/good/blue_good_2020-11-18_02-35-31.737419.txt
0 0.506696 0.361607 0.852679 0.428571
1 0.502232 0.448661 0.879464 0.772321

['1', '0.502232', '0.448661', '0.879464', '0.772321']
2 /content/darknet/data/obj/good/green_good_2020-11-18_03-11-13.341336.txt
0 0.502232 0.448661 0.879464 0.772321
1 0.517857 0.234375 0.723214 0.415179

['1', '0.517857', '0.234375', '0.723214', '0.415179']
3 /content/darknet/data/obj/good/lblue_good_2020-11-18_03-27-35.164558.txt
0 0.517857 0.234375 0.723214 0.415179
1 0.477679 0.383929 0.696429 0.348214

['1', '0.477679', '0.383929', '0.696429', '0.348214']
4 /content/darknet/data/obj/good/white_good_2020-11-18_01-19-40.339647.txt
0 0.47767

In [11]:
# import shutil
# shutil.rmtree('/content/darknet/data/obj/')

In [15]:
import glob
images_list = glob.glob("data/obj/*/*.jp*")
print(images_list)

['data/obj/good/white_good_2020-11-18_01-20-04.040680.jpg', 'data/obj/good/blue_good_2020-11-18_02-35-28.246355.jpg', 'data/obj/good/blue_good_2020-11-18_02-35-47.721610.jpg', 'data/obj/good/black_good_2020-11-18_01-35-36.220566.jpg', 'data/obj/good/blue_good_2020-11-18_02-35-37.413964.jpg', 'data/obj/good/yellow_good_2020-11-18_18-22-17.157955.jpg', 'data/obj/good/yellow_good_2020-11-18_18-21-45.779543.jpg', 'data/obj/good/lblue_good_2020-11-18_03-27-48.586508.jpg', 'data/obj/good/white_good_2020-11-18_01-19-53.505577.jpg', 'data/obj/good/yellow_good_2020-11-18_18-22-07.602293.jpg', 'data/obj/good/gray_good_2020-11-18_04-28-20.254534.jpg', 'data/obj/good/white_good_2020-11-18_01-19-37.945508.jpg', 'data/obj/good/white_good_2020-11-18_01-20-17.355212.jpg', 'data/obj/good/green_good_2020-11-18_03-11-20.573551.jpg', 'data/obj/good/blue_good_2020-11-18_02-35-31.737419.jpg', 'data/obj/good/black_good_2020-11-18_01-34-55.109395.jpg', 'data/obj/good/lblue_good_2020-11-18_03-27-52.509288.jpg'

In [16]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.004922, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.559400, iou_loss = 0.000000, total_loss = 0.559400 
 total_bbox = 15460, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.282685, GIOU: 0.216625), Class: 0.567973, Obj: 0.040834, No Obj: 0.023258, .5R: 0.000000, .75R: 0.000000, count: 4, class_loss = 1.824613, iou_loss = 2.195340, total_loss = 4.019953 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.010159, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.541165, iou_loss = 0.000000, total_loss = 0.541165 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj:

In [ ]:
# start training from the last saved weights
# !./darknet detector train data/obj.data cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights -dont_show

In [ ]:
%cd cfg
!sed -i 's/batch=64batch=1' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=16/subdivisions=1' cfg/yolov3_training.cfg
%cd ..

/content/darknet/cfg
sed: -e expression #1, char 17: unterminated `s' command
sed: -e expression #1, char 32: unterminated `s' command
/content/darknet
